In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df1 = pd.read_csv('D:/Curso Henry/Proyecto final/CSV_Limpiados/Copia de SalesFINAL12312016_Modificado.csv')
df2 = pd.read_csv('D:/Curso Henry/Proyecto final/CSV_Limpiados/Copia de PurchasePricesDec2017_Modificado.csv')

In [3]:
claves_producto = ['Brand', 'Description', 'Size']

# Limpiar textos
for col in claves_producto:
    df1[col] = df1[col].astype(str).str.strip()
    df2[col] = df2[col].astype(str).str.strip()

In [4]:
# Claves de productos en df2
productos_df2 = df2[claves_producto].drop_duplicates()

# Claves de productos en df1
productos_df1 = df1[claves_producto].drop_duplicates()

# Filas que están en df1 pero no en df2
nuevos_productos = productos_df1.merge(productos_df2, on=claves_producto, how='left', indicator=True)
nuevos_productos = nuevos_productos[nuevos_productos['_merge'] == 'left_only'].drop(columns=['_merge'])

In [5]:
datos_nuevos = df1.merge(nuevos_productos, on=claves_producto, how='inner')

# Agrupar para obtener una fila por producto
nuevas_filas = datos_nuevos.groupby(claves_producto, as_index=False).agg({
    'SalesPrice': 'mean',
    'Volume': 'first',
    'Classification': 'first',
    'VendorNo': 'first'
})

# Renombrar columnas para que coincidan con df2
nuevas_filas.rename(columns={
    'SalesPrice': 'Price',
    'VendorNo': 'VendorNumber'
}, inplace=True)

# Añadir columnas que le faltan para estar igual que df2
for col in df2.columns:
    if col not in nuevas_filas.columns:
        nuevas_filas[col] = np.nan

# Ordenar columnas igual que df2
nuevas_filas = nuevas_filas[df2.columns]

In [6]:
df2_completo = pd.concat([df2, nuevas_filas], ignore_index=True)

In [7]:
print(f"Cantidad original en df2: {len(df2)}")
print(f"Nuevas filas agregadas: {len(nuevas_filas)}")
print(f"Total final en df2_completo: {len(df2_completo)}")

Cantidad original en df2: 6129
Nuevas filas agregadas: 6101
Total final en df2_completo: 12230


In [9]:
df2_completo.head()

,Brand,Description,Price,Size,Volume,Classification,PurchasePrice,VendorNumber,VendorName,fecha_actualizacion
0,1,Frizze Blue 1000mL,11.50,1000mL,1000.0,0,11.50,11500,Distribuidora Sur S.A.,2025-05-30 15:14:13
1,2,Cynar 750mL,13.99,750mL,750.0,0,13.99,11501,Importadora Global Ltda.,2025-05-30 15:14:13
2,4,Frizze Blue 1000mL,11.50,1000mL,1000.0,0,11.50,11500,Distribuidora Sur S.A.,2025-05-30 15:19:54
3,5,Cynar 750mL,13.99,750mL,750.0,0,13.99,11501,Importadora Global Ltda.,2025-05-30 15:19:54
4,61,Margaritaville Silver,13.99,750mL,750.0,1,10.60,8004,SAZERAC CO INC,2025-05-27 18:54:32


In [10]:
columnas_a_eliminar = ['Size', 'fecha_actualizacion', 'Classification']

df2_completo = df2_completo.drop(columns=[col for col in columnas_a_eliminar if col in df2_completo.columns])

In [11]:
columnas_coma = ['Price', 'PurchasePrice']

for col in columnas_coma:
    df2_completo[col] = df2_completo[col].astype(str).str.replace('.', ',', regex=False)

In [12]:
#Ya hicimos toda la limpieza, guardamos
df2_completo.to_csv('Productos.csv', index=False)